In [1]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

# import ind.alpha191 as a191


import matplotlib.pyplot as plt
import seaborn as sns

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates



# from QUANTAXIS.QAUtil import DATABASE
# from QUANTAXIS.QAUtil import  trade_date_sse
# from QUANTAXIS.QAUtil.QADate_trade import (
#     QA_util_get_pre_trade_date,
#     QA_util_get_next_trade_date,
#     QA_util_if_tradetime
# )

# import inspect

# from sklearn import linear_model

%load_ext autoreload
%autoreload 2
%aimport ind.alpha191

In [2]:
###加载数据

ids = np.arange(1,192)
# np.random.shuffle(ids)
files = list(map(lambda x:'191_alpha'+'0'*(3- int(np.log10(x)+1))+str(x)+'_train',ids))

def load_alpha_factors(file_names):
    from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
    from base.JuUnits import excute_for_multidates
    import numpy as np
    import pandas as pd 
    
    temp = []
    for file in file_names:
        print(file)
        temp.append(load_cache(file, cache_type=CACHE_TYPE.FACTOR))
    
    return pd.concat(temp,axis=1)
        
worker=6
task = task_chunk_split(files, worker)
results = parallal_task(worker, load_alpha_factors, task)
res = pd.concat(results,axis=1)

Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:21644  pid:25024


  0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
def filter_invalid_factor(fators_df):
    ## 缺失30%以上，且3倍方差以上，废弃
    ## 加极端判断是为了保证留下那种大窗口期的因子。
    def nan_count(x):
        return round(x.isna().sum(axis=0)/len(x),4)*100
    
    #计算缺失率
    valid_rate = excute_for_multidates(res, nan_count, level=1) 
    valid_distribute_boundary = valid_rate.mean(axis=0) + valid_rate.std(axis=0)*3
    #缺失率符合当前因子在所有标的上，符合3个标准差的
    filter_contition = valid_rate > valid_distribute_boundary 
    
    def filter_invalid(x):
        code = x.index.get_level_values(1)[0]
        distribute_condition = filter_contition.loc[code] 
        #符合3个标准差的列
        invalid_factors_name = distribute_condition[distribute_condition==True].index 
        #符合3个标准差，并且缺失30%以上的列
        thredhold_condition = valid_rate.loc[code][invalid_factors_name]>30 
        
        x[thredhold_condition.index] = np.nan 
        return x

    filtered = excute_for_multidates(fators_df, filter_invalid, level=1)
    
    return filtered

factors_filtered = filter_invalid_factor(res)

In [4]:
def filter_correlative_factor(fators_df, thredhold=.70, show_chart=False,verbose=False):
    #去掉相关性强的因子，相关性大于thredhold的因子（列）将被去除
    corr = fators_df.corr()
    matrix = corr.values
    #取得下三角矩阵
    matrix[np.triu_indices(matrix.shape[0])]=np.nan
    corr_triu = pd.DataFrame(matrix, columns=corr.columns, index=corr.index)

    if show_chart:
        plt.figure(figsize=(30,30))
        labels = corr.columns.str.replace('191_alpha','')
        ax = sns.heatmap(corr_triu, xticklabels=labels, yticklabels=labels, cmap="YlGnBu",cbar=True)

    # 取得符合条件的因子对后，再取“一对的左边”作为删除目标
    correlative_columns_name = corr_triu.abs().gt(thredhold).stack().loc[lambda x: x].index.get_level_values(0).unique().to_list()
    
    if verbose:
        print('delete correlative factors: {}'.format(','.join(correlative_columns_name)))
        
    correlative_droed = fators_df.drop(columns=correlative_columns_name)
    return correlative_droed

factors_filtered = filter_correlative_factor(factors_filtered,show_chart=False,verbose=True)

delete correlative factors: 191_alpha008,191_alpha014,191_alpha020,191_alpha022,191_alpha024,191_alpha026,191_alpha027,191_alpha028,191_alpha031,191_alpha034,191_alpha037,191_alpha040,191_alpha046,191_alpha047,191_alpha049,191_alpha050,191_alpha051,191_alpha052,191_alpha053,191_alpha055,191_alpha057,191_alpha058,191_alpha059,191_alpha062,191_alpha065,191_alpha066,191_alpha067,191_alpha068,191_alpha071,191_alpha072,191_alpha078,191_alpha079,191_alpha082,191_alpha088,191_alpha089,191_alpha090,191_alpha094,191_alpha095,191_alpha096,191_alpha097,191_alpha099,191_alpha100,191_alpha105,191_alpha106,191_alpha112,191_alpha116,191_alpha121,191_alpha122,191_alpha126,191_alpha128,191_alpha129,191_alpha132,191_alpha133,191_alpha134,191_alpha136,191_alpha137,191_alpha139,191_alpha143,191_alpha147,191_alpha150,191_alpha153,191_alpha154,191_alpha156,191_alpha160,191_alpha161,191_alpha162,191_alpha165,191_alpha167,191_alpha168,191_alpha173,191_alpha174,191_alpha175,191_alpha177,191_alpha180,191_alpha1

In [38]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):

factors = factors_filtered

codes = factors.index.get_level_values(1).unique()
dates = factors.index.get_level_values(0).unique()
data_replenish = smpl.get_data(codes.to_list(),dates.min().strftime('%Y-%m-%d'),dates.max().strftime('%Y-%m-%d')).data
ret_forward = smpl.get_forward_return(data_replenish,'close')
ret_forward.name = 'ret_forward'


mv_industry = smpl.add_marketvalue_industry(data_replenish).drop(columns=data_replenish.columns.to_list())
industy = mv_industry['industry']
cap = mv_industry['totalCapital']


# factors = factors.assign(cap=cap,industry = industy)
factors = factors.assign(country=1.0, cap=cap,industry = industy)
factors = pd.get_dummies(factors,'industry')

## 国家因子换到第一列
columns_ = factors.columns.to_list()
columns_.insert(0,columns_.pop(columns_.index('country')))
factors = factors.reindex(columns=columns_)



# adj_weight = np.sqrt(cap)/np.sqrt(cap).sum()
# adj_weight_mat = pd.DataFrame(np.diag(adj_weight),index=adj_weight.index, columns=adj_weight.index)

# del factors['country']



In [155]:
# factors.columns.to_list()

test = factors.loc['2017-01-04']
test_industy_dummy = test.iloc[:,-1*len(industry_wight):]
test_cap=test['cap']

##个股权重矩阵，权重为流通市值平方占比
cap_temp = np.sqrt(test_cap)
wight = cap_temp/cap_temp.sum()
V = np.diag(wight) #pd.DataFrame(np.diag(wight),columns = test_cap.index,index = test_cap.index)


industry_cap = test_cap.T.dot(test_industy_dummy)/industry_cap.sum()
industry_wight = industry_cap/industry_cap.sum()


test_train = test[test.columns.difference(['cap'],sort=False)]

k = test_train.shape[1] #总因子数量
p = len(industry_wight) #行业因子数量
# q = k-1-p #风格因子数量
q = k-p #原风格因子数量


diag_R = np.diag(np.ones(k))
location = q
R = np.delete(diag_R,location+p-1,axis = 1)
adj_industry_weights = -industry_wight.div(industry_wight[-1]).iloc[:-1]
R[location+p-1,location:location+p] = adj_industry_weights


showR = pd.DataFrame(R,columns=np.delete(test_train.columns,location+p-1),index=test_train.columns)
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(showR)
    
showR.to_excel('./showR.xlsx')

,country,191_alpha001,191_alpha002,191_alpha003,191_alpha004,191_alpha005,191_alpha006,191_alpha007,191_alpha009,191_alpha010,191_alpha011,191_alpha012,191_alpha013,191_alpha015,191_alpha016,191_alpha017,191_alpha018,191_alpha019,191_alpha021,191_alpha023,191_alpha025,191_alpha029,191_alpha030,191_alpha032,191_alpha033,191_alpha035,191_alpha036,191_alpha038,191_alpha039,191_alpha041,191_alpha042,191_alpha043,191_alpha044,191_alpha045,191_alpha048,191_alpha054,191_alpha056,191_alpha060,191_alpha061,191_alpha063,191_alpha064,191_alpha069,191_alpha070,191_alpha073,191_alpha074,191_alpha075,191_alpha076,191_alpha077,191_alpha080,191_alpha081,191_alpha083,191_alpha084,191_alpha085,191_alpha086,191_alpha087,191_alpha091,191_alpha092,191_alpha093,191_alpha098,191_alpha101,191_alpha102,191_alpha103,191_alpha104,191_alpha107,191_alpha108,191_alpha109,191_alpha110,191_alpha111,191_alpha113,191_alpha114,191_alpha115,191_alpha117,191_alpha118,191_alpha119,191_alpha120,191_alpha123,191_alpha124,191_alpha125,191_alpha127,191_alpha130,191_alpha131,191_alpha135,191_alpha138,191_alpha140,191_alpha141,191_alpha142,191_alpha144,191_alpha145,191_alpha146,191_alpha148,191_alpha149,191_alpha151,191_alpha152,191_alpha155,191_alpha157,191_alpha158,191_alpha159,191_alpha163,191_alpha164,191_alpha166,191_alpha169,191_alpha170,191_alpha171,191_alpha172,191_alpha176,191_alpha178,191_alpha179,191_alpha181,191_alpha182,191_alpha184,191_alpha186,191_alpha188,191_alpha190,191_alpha191,industry_PCB,industry_中药,industry_乳制品,industry_云软件服务,industry_产联网信息服务,industry_住宅开发,industry_保险,industry_光伏电池组件,industry_光伏硅料,industry_光伏硅片,industry_光伏辅材,industry_其他IT设备,industry_其他专业工程,industry_其他家居,industry_其他广告营销,industry_其他建材,industry_其他酒类,industry_军工电子,industry_农商行,industry_冰洗,industry_功率半导体,industry_动力煤,industry_化学制剂,industry_化学工程,industry_医疗研发外包,industry_医院,industry_半导体封测,industry_半导体设备,industry_原料药,industry_厨房小家电,industry_品牌消费电子,industry_商业物业经营,industry_商用载货车,industry_啤酒,industry_国有大型银行,industry_城商行,industry_基因药物,industry_基建工程,industry_安防设备,industry_家具,industry_家电零部件,industry_工程机械整机,industry_工程机械零件,industry_底盘与发动机系统,industry_快递,industry_房屋建设,industry_教育培训,industry_文化用品,industry_新兴软件,industry_旅游服务,industry_有机化工原料,industry_有机硅,industry_机场,industry_板材,industry_核力发电,industry_氨纶,industry_氯碱,industry_水产饲料,industry_水力发电,industry_水泥制造,industry_汽车电子电气系统,industry_油气开采,industry_油田服务,industry_消费电子组件,industry_涤纶,industry_清洁小家电,industry_港口,industry_游戏,industry_激光设备,industry_火力发电,industry_炼油化工,industry_煤化工,industry_特钢,industry_玻纤制造,industry_生猪养殖,industry_电信运营服务,industry_电动乘用车,industry_电池化学品,industry_电网自动化,industry_畜禽饲料,industry_白酒,industry_稀土,industry_空调,industry_系统设备,industry_纺织化学品,industry_综合乘用车,industry_聚氨酯,industry_肉制品,industry_股份制银行,industry_航天装备,industry_航海装备,industry_航空装备,industry_航空运输,industry_航运,industry_血液制品,industry_行业应用软件,industry_证券,industry_诊断服务,industry_诊断试剂,industry_调味品,industry_车身附件及饰件,industry_轨交设备,industry_轮胎轮毂,industry_软饮料,industry_输变电设备,industry_配电设备,industry_钛白粉,industry_钨钼,industry_钴,industry_钾肥,industry_铁路运输,industry_铜,industry_铝,industry_锂,industry_锂电池,industry_防水材料,industry_集成电路设计,industry_面板,industry_风力发电,industry_风电整机
country,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [159]:
#     def calc_R(ind,factor):
#         #计算约束矩阵防止共线性问题
#         #获取申万28个行业列表
#         _sw_ind = list(ind.drop_duplicates().values)
        
#         ind_float_mv = pd.DataFrame(index = stocks,columns = ['industry','float_mv'])
#         ind_float_mv['industry'] = ind
#         ind_float_mv['float_mv'] = _float_mv.T.loc[stocks,date]
#         _sum_float_mv = ind_float_mv['float_mv'].sum()
        
#         _industry_wgt = pd.Series(index = _sw_ind)
        
#         for i in _sw_ind:
#             #计算行业流通市值占比
#             _industry_wgt[i] = ind_float_mv[ind_float_mv['industry']==i]['float_mv'].sum()/_sum_float_mv
#         #构建约束矩阵R，R为K*K-1矩阵，K为因子数量
#         #将最后一个行业因子表示为其他行业的线性形式
#         #移除最后一个行业所在的列，并对其所在的行进行权重处理
#         k = factor.shape[1] #总因子数量
#         p = len(_industry_wgt) #行业因子数量
#         q = k-1-p #风格因子数量
        
#         diag_R = np.diag(np.ones(k))
#         location = len(_industry_wgt)
#         R = np.delete(diag_R,location,axis = 1)
#         adj_industry_weights = -_industry_wgt.div(_industry_wgt[-1]).iloc[:-1]
#         R[location,1:p] = adj_industry_weights
        
#         return R
# np.delete(test_train.columns,location).to_list()
# adj_industry_weights= -industry_wight.div(industry_wight[-1]).iloc[:-1]

# adj_industry_weights
test_industy_dummy['industry_风力发电']
# industry_cap

code
000001    0
000002    0
000063    0
000066    0
000069    0
         ..
603369    0
603799    0
603806    0
603899    0
603993    0
Name: industry_风力发电, Length: 202, dtype: uint8